In [5]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset
import torch.nn as nn
import cv2
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
model.classifier = nn.Identity()
model.eval()
image = cv2.imread('./3.jpg')
inputs = image_processor(image, return_tensors="pt")
print(model(**inputs).logits.flatten().detach().numpy().shape)

(768,)


In [3]:
query_path = 'query_images/'
query_files = os.listdir(query_path)
print(len(query_files))

retrieval_path = 'images/'
retrieval_files = os.listdir(retrieval_path)
print(len(retrieval_files))

print(query_files[:5])

200
10000
['64d323101c620faef16a9afd_6_agstore_vn.jpg\n', '64da9bac1c3e4f08a26d57d8_2_chicland_vn.jpg\n', '64d25998c0c243fd215f711f_0_hhluxury_vn.jpg\n', '64da8ec21c3e4f08a26d43a6_1_chicland_vn.jpg\n', '64d3238a1c620faef16a9beb_5_agstore_vn.jpg\n']


In [8]:
# n_queries = len(query_files)
# query_matrix = np.zeros((n_queries, 768))
# for i in tqdm(range(n_queries)):
#     img = Image.open(query_path+query_files[i]).convert('RGB')
#     inputs = image_processor(img, return_tensors="pt")
#     vector = model(**inputs).logits.flatten().detach().numpy()
#     query_matrix[i, :] = vector    
# np.save('ViT_query_matrix.npy', query_matrix)
# print(query_matrix.shape)
# print(query_matrix)

100%|██████████████████████████████████████████████████| 200/200 [00:48<00:00,  4.15it/s]

(200, 768)
[[-0.27993301 -0.60705513 -0.13782115 ... -1.18001032 -0.33467326
   0.1083511 ]
 [-1.7129395  -1.52261174 -1.22056258 ...  0.29758045 -0.26965815
   0.39323455]
 [-2.15507483 -1.01523161 -0.9915843  ...  0.57583344 -0.82860291
  -0.614016  ]
 ...
 [-0.54857415 -0.74036086 -0.56872332 ...  0.60176349 -1.03259015
   0.24519567]
 [-0.65177894  0.1204956  -0.62647688 ...  0.58632255 -0.80400562
   0.40110806]
 [-0.13847269 -2.023314   -0.02394438 ... -1.09086251  0.0556707
  -1.26499426]]


In [9]:
# n_retrievals = len(retrieval_files)
# retrieval_matrix = np.zeros((n_retrievals, 768))
# for i in tqdm(range(n_retrievals)):
#     try:
#         img = Image.open(retrieval_path+retrieval_files[i]).convert('RGB')
#         inputs = image_processor(img, return_tensors="pt")
#         vector = model(**inputs).logits.flatten().detach().numpy()
#         retrieval_matrix[i, :] = vector
#     except:
#         retrieval_matrix[i, :] = np.zeros((1, 768))
# np.save('ViT_retrieval_matrix.npy', retrieval_matrix)
# print(retrieval_matrix.shape)
# retrieval_matrix
    

 78%|████████████████████████████████████▊          | 7844/10000 [32:36<11:15,  3.19it/s]/home/binhng/anaconda3/envs/pt/lib/python3.11/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████████████████████████████████████████| 10000/10000 [41:34<00:00,  4.01it/s]

(10000, 768)


array([[ 0.38280964,  0.32109451, -0.04591281, ..., -0.61909074,
        -0.66329128,  0.11308269],
       [-1.12061608, -0.67885423, -1.09242678, ..., -0.72129434,
        -0.59230292,  0.79317075],
       [ 0.11617942,  0.04850836,  1.36921823, ..., -0.29227048,
         0.34549344,  0.92147171],
       ...,
       [ 0.67619127, -0.64255548,  0.77771777, ..., -1.92345464,
         0.27126929,  0.96744001],
       [-1.21616817, -0.27696335, -2.19197702, ..., -0.46903911,
        -0.04838647,  0.89184904],
       [-1.55875397, -0.23155649,  0.84143639, ...,  2.01766014,
        -1.2676903 ,  1.10442913]])

In [29]:
def calc_Euclid_dist(B1, B2):
    # B1 = query_matrix
    # B2 = retrieval_matrix
    distances = np.zeros((B1.shape[0], B2.shape[0]))
    for i in range(200):
        distances[i, :] = np.sum((B1[i] - B2)**2, axis = 1)
    distances = distances**0.5
    return distances
def get_topk(B1, B2, topk): #get topk idx
    ind = np.argsort(calc_Euclid_dist(B1, B2))
    ids = np.array(range(B2.shape[0]))
    ids = ids[ind]
    return ids[:, :topk]
def get_L2(B1, B2, topk): # get topk labels
    ids = list(get_topk(B1, B2, topk))
    topk_labels = retrieval_labels[ids]
    # L2 = [[img_vector_lst[j][0] for j in i] for i in ids] #[[arr[j] for j in i] for i in idx]
    # print(topk_labels)
    return topk_labels

def calc_topk_mAP_2(B1, B2, L1, topk): 
    '''
    B1: query matrix
    B2: retrieval matrix
    L1: labels of query images
    topk: k
    '''
    topk_retrievals = get_L2(B1, B2, topk)
    topk_mAP = 0
    num_query = B1.shape[0]
    # fig, axes = plt.subplots(num_query, topk+1, figsize=(12, 8))
    for iter in tqdm(range(num_query)):
        true_label = L1[iter].split('_')
        true_idx = []
        # axes[iter, 0].imshow(plt.imread('query_images/'+L1[iter]))
        # axes[iter, 0].axis('off')
        for i in range(topk):
            # axes[iter, i+1].imshow(plt.imread('images/'+topk_retrievals[iter][i]))
            # axes[iter, i+1].axis('off')
            if topk_retrievals[iter][i].split('_')[0] == true_label[0]:
                true_idx.append(i+1)
        true_idx = np.array(true_idx)
        # true_idx = (np.asarray(np.where(L2[ids][iter, :]==true_label)) + 1).ravel()
        # print('true_idx', true_idx)
        if true_idx.shape[0] == 0:
            continue
        count = np.linspace(1, true_idx.shape[0], true_idx.shape[0])
        # print('count', count)
        topk_mAP_ = np.mean(count/true_idx)
        # print('topk_mAP_:', topk_mAP_)
        # print('==========================================')
        topk_mAP += topk_mAP_
    # plt.tight_layout()  # Ensure subplots do not overlap
    # plt.show()
    topk_mAP /= num_query
    # print('=====================================================================')
    # print('topk_mAP', topk_mAP)
    return topk_mAP 

In [30]:
query_matrix = np.load('ViT_query_matrix.npy')
query_labels = np.load('query_labels.npy')
retrieval_matrix = np.load('ViT_retrieval_matrix.npy')
retrieval_labels = np.load('retrieval_labels.npy')
query_matrix.shape, query_labels.shape, retrieval_matrix.shape, retrieval_labels.shape

((200, 768), (200,), (10000, 768), (10000,))

In [31]:
error_idx = [2749, 3952, 4893, 5296, 6288, 8028, 8463, 8735, 9697]
retrieval_matrix =  np.delete(retrieval_matrix , error_idx, axis=0) 
retrieval_labels = np.delete(retrieval_labels , error_idx, axis=0) 
retrieval_matrix.shape, retrieval_labels.shape

((9991, 768), (9991,))

In [33]:
topk_lst = [1, 5, 10, 20, 100, 1000,retrieval_labels.shape[0]]
print('ViT mAP: ')
for i in topk_lst:
    print(f'mAP@{i}:', calc_topk_mAP_2(query_matrix, retrieval_matrix, query_labels, i))

ViT mAP: 


100%|██████████████████████████████████████████████| 200/200 [00:00<00:00, 126220.40it/s]


mAP@1: 0.59


100%|███████████████████████████████████████████████| 200/200 [00:00<00:00, 83427.23it/s]


mAP@5: 0.6746874999999997


...

mAP@100: 0.5486334198563292


100%|████████████████████████████████████████████████| 200/200 [00:00<00:00, 2505.46it/s]


mAP@1000: 0.47086394329307507


100%|█████████████████████████████████████████████████| 200/200 [00:00<00:00, 252.02it/s]

mAP@9991: 0.37759100946225294
